# SSDlite

In [ ]:
import torch
import torchvision
from torchvision.transforms import v2
from data import *

In [ ]:
def get_weight_filename(epoch: int):
    return f"ssdlite_epoch{str(epoch).zfill(4)}.pth"

def build_model(num_classes: int, resume: str, device: str = "cpu"):
    ### Instantiate model
    model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(
        weights=torchvision.models.detection.SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
    )

    ### Replace the head of the network
    if num_classes:
        # First get some parameters of the original classification head
        in_channels = [672,480,512,256,256,128]
        num_anchors = [6,6,6,6,6,6]
        # Build a new classification head
        model.head = torchvision.models.detection.ssd.SSDHead(
            in_channels=in_channels,
            num_anchors=num_anchors,
            num_classes=num_classes+1,
        )

    ### Load custom weights
    if resume:
        model.load_state_dict(torch.load(resume))
    return model.to(device)



def train_one_epoch(model, optim, device="cpu") -> dict:
    # Initialize epoch losses
    epoch_loss = 0
    epoch_loss_classification = 0
    epoch_loss_bbox_regression = 0
    
    for data in train_dataloader:
        images, targets = extract_images_targets(data, device=device)
        ### Compute loss of batch
        loss_dict = model(images, targets)
        loss_classification = loss_dict['classification'].detach().cpu().numpy()
        loss_bbox_regression = loss_dict['bbox_regression'].detach().cpu().numpy()
        loss = sum(v for v in loss_dict.values())
        loss_total = loss.detach().cpu().numpy()
        ### Update epoch losses
        epoch_loss += loss_total
        epoch_loss_classification += loss_classification
        epoch_loss_bbox_regression += loss_bbox_regression
        ### Backpropagation
        optim.zero_grad()
        loss.backward()
        optim.step()
    return {
        "loss": epoch_loss,
        "loss_classification": epoch_loss_classification,
        "loss_bbox_regression": epoch_loss_bbox_regression,
    }
    

def train(n_epochs: int, lr: float, train_dataloader, start_epoch: int = 0, resume: str = None, save_every: int = None, num_classes = None, device="cpu"):
    # check if custom weight file is available
    if start_epoch and start_epoch > 0:
        resume = get_weight_filename(start_epoch)
    else:
        resume = False
    
    # Instantiate model
    model = build_model(num_classes=num_classes, resume=resume, device=device)
    
    # Optimizer
    optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=3, gamma=0.1)

    model.train()
    print("Start training ...")

    for epoch in range(start_epoch+1, n_epochs+1):
        # Train the model for one epoch
        epoch_metrics = train_one_epoch(model, optim, device=device)

        # Update learning rate
        lr_scheduler.step()
        print(f"Epoch {epoch}/{n_epochs}: loss={epoch_metrics['loss']}, loss_classification={epoch_metrics['loss_classification']}, loss_bbox_regression={epoch_metrics['loss_bbox_regression']}")
        if save_every and epoch > 0 and epoch % save_every == 0:
            torch.save(model.state_dict(), get_weight_filename(epoch))

In [ ]:
root_dir = "C:\\Users\\tilof\\PycharmProjects\\DeepLearningProjects\\DETR\\data\\spine"
train_data_dir = f"{root_dir}\\train2017"
train_annotation_file = f"{root_dir}\\annotations\\instances_train2017.json"
n_epochs = 200
start_epoch = 0
num_classes = 1
lr = 1e-3
batch_size = 128
save_every = 10

# Dataset
transforms = v2.Compose([
    v2.ToTensor(),
    v2.ToDtype(torch.float32),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = build_coco_dataset(root=train_data_dir, annFile=train_annotation_file, transform=transforms)
train_dataloader = build_dataloader(dataset=train_dataset, batch_size=batch_size, collate_fn=collate_fn_coco)

device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device", device)

In [ ]:
train(
    n_epochs=n_epochs,
    lr=lr,
    train_dataloader=train_dataloader,
    start_epoch=start_epoch,
    save_every=save_every,
    num_classes=num_classes,
    device=device,
)